### O que é PCA?

O PCA (Principal Component Analysis) é uma técnica matemática que transforma um conjunto de **muitas variáveis** em um número menor de **componentes principais**, preservando ao máximo a **informação dos dados**.

Exemplo intuitivo:
- Imagine que temos um conjunto de imagens de gatos e queremos encontrar um jeito de representar essas imagens com **menos variáveis**, sem perder muita informação. O PCA encontra as características mais importantes das imagens e reduz os dados para que ainda seja possível reconhecer os gatos, mas com menos detalhes.

#### Quando o PCA pode melhorar ou piorar um modelo?

**Quando o PCA pode melhorar o desempenho?**
- Reduz overfitting -> Se os dados têm muitas colunas irrelevantes, o PCA pode remover ruído e melhorar a generalização do modelo.
- Diminui a complexidade -> Modelos treinam mais rápido porque há menos variáveis.
- Reduz correlações entre variáveis -> Modelos como regressão linear funcionam melhor com variáveis menos correlacionadas, podendo reduzir a redundância entre elas.

**Quando o PCA pode piorar o desempenho?**
- Perda de informação -> Se muitos componentes principais forem removidos, o modelo pode perder detalhes importantes.
- Dificulta interpretação -> Os novos componentes não têm significado direto (não são "idade", "altura", etc. mas sim combinações dessas variáveis).
- Pode não ser necessário -> Se os dados já têm poucas dimensões ou as variáveis são muito informativas, o PCA pode atrapalhar mais do que ajudar.

#### Como saber se deve usar o PCA?
- Se você tem muitas variáveis (muito maior que o número de amostras), o PCA pode ajudar.
- Se as variáveis são altamente correlacionadas, o PCA pode reduzir redundância.
- Se o modelo está demorando muito para treinar, PCA pode acelerar o processamento.
- Se o modelo perde precisão após o PCA, talvez ele esteja removendo informações importantes.

**Dica prática:** Experimente rodar o modelo **com e sem PCA** e compare os resultados.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA # Redução de dimensionalidade
from sklearn.metrics import mean_squared_error # Erro quadrático médio
from sklearn.preprocessing import StandardScaler # Padronização

In [2]:
df = pd.read_csv('crop_yield.csv', sep=',')
df.head()

,Crop,Precipitation (mm day-1),Specific Humidity at 2 Meters (g/kg),Relative Humidity at 2 Meters (%),Temperature at 2 Meters (C),Yield
0,"Cocoa, beans",2248.92,17.72,83.40,26.01,11560
1,"Cocoa, beans",1938.42,17.54,82.11,26.11,11253
2,"Cocoa, beans",2301.54,17.81,82.79,26.24,9456
3,"Cocoa, beans",2592.35,17.61,85.07,25.56,9321
4,"Cocoa, beans",2344.72,17.61,84.12,25.76,8800


In [3]:
df['Crop'].value_counts()

Crop
Cocoa, beans       39
Oil palm fruit     39
Rice, paddy        39
Rubber, natural    39
Name: count, dtype: int64

**Pré processamento**

In [4]:
# Converte dados categóricos (coluna Crop) em valores numéricos usando one-hot encoding
df = pd.get_dummies(df,
                    columns=['Crop'],
                    prefix=['Crop'])
df.head()

,Precipitation (mm day-1),Specific Humidity at 2 Meters (g/kg),Relative Humidity at 2 Meters (%),Temperature at 2 Meters (C),Yield,"Crop_Cocoa, beans",Crop_Oil palm fruit,"Crop_Rice, paddy","Crop_Rubber, natural"
0,2248.92,17.72,83.40,26.01,11560,True,False,False,False
1,1938.42,17.54,82.11,26.11,11253,True,False,False,False
2,2301.54,17.81,82.79,26.24,9456,True,False,False,False
3,2592.35,17.61,85.07,25.56,9321,True,False,False,False
4,2344.72,17.61,84.12,25.76,8800,True,False,False,False


Supondo que a coluna 'yield' é a variável dependente (target), iremos separar os dados em X (features) e y (target).

In [5]:
X = df.drop('Yield', axis=1)
y = df['Yield']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=70)

Reescalando os dados com padronização (StandardScaler).

In [6]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Modelo de regressão **sem PCA**.

In [7]:
regressor = LinearRegression()
regressor.fit(X_train_scaled, y_train)

# Prevê e calcula o erro (sem PCA)
y_pred_no_pca = regressor.predict(X_test_scaled)
mse_no_pca = mean_squared_error(y_test, y_pred_no_pca)

print(f'Erro Quadrático Médio sem PCA: {mse_no_pca}')

Erro Quadrático Médio sem PCA: 107381321.15863115


Aplicar PCA para reduzir a dimensionalidade antes do treinamento.

In [8]:
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

Modelo de regressão **com PCA**.

In [9]:
regressor = LinearRegression()
regressor.fit(X_train_pca, y_train)

# Prevê e calcula o erro (sem PCA)
y_pred_pca = regressor.predict(X_test_pca)
mse_pca = mean_squared_error(y_test, y_pred_pca)

print(f'Erro Quadrático Médio com PCA: {mse_pca}')

# Comparando resultados

if mse_no_pca <= mse_pca:
    print('O modelo sem PCA teve um desempenho melhor.')
else:
    print("O modelo com PCA teve um desempenho melhor.")

Erro Quadrático Médio com PCA: 6200965625.459932
O modelo sem PCA teve um desempenho melhor.
